<a href="https://colab.research.google.com/github/darkwings/ai-notebooks/blob/main/ReAct_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct agents

A simple ReAct agent with LangGraph



In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community langgraph

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


Let's import all the necessary stuff

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
    AIMessage
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
import os

Now we can implement the needed classes.

Let's start with the prompt

In [5]:
# Define a more structured prompt template with tool descriptions
prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
Thought: {agent_scratchpad}"""


Then, the State and the tool. We simulate a tool that returns the rating of some restaurants

In [6]:
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

# This is the tool
# The tool could be implemented also as a function, in this case
# we use a class. LangGraph will call the __call__ function
class RestaurantTool:
    def __init__(self):
        self.name = "restaurant_rating"
        self.description = "Get rating and review information for a restaurant"

    def get_restaurant_rating(self, name: str) -> dict:
        ratings = {
            "Pizza Palace": {"rating": 4.5, "reviews": 230},
            "Burger Barn": {"rating": 4.2, "reviews": 185},
            "Sushi Supreme": {"rating": 4.8, "reviews": 320}
        }
        return ratings.get(name, {"rating": 0, "reviews": 0})

    def __call__(self, name: str) -> str:
        result = self.get_restaurant_rating(name)
        return f"Rating: {result['rating']}/5.0 from {result['reviews']} reviews"

## Agent class

Now the agent class

In [10]:
class ReActAgent:
    def __init__(self, model: ChatOpenAI, tools: List[Tool], system: str = ''):
        self.system = system
        self.tools = {t.name: t for t in tools}

        # Create tool descriptions for the prompt
        tool_descriptions = "\n".join(f"- {t.name}: {t.description}" for t in tools)
        tool_names = ", ".join(t.name for t in tools)

        # Bind tools to the model
        self.model = model.bind_tools(tools)

        # Initialize the graph
        graph = StateGraph(AgentState)

        # Add nodes and edges
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)

        # Add conditional edges
        graph.add_conditional_edges(
            "llm",
            self.should_continue,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")

        # Set entry point and compile
        graph.set_entry_point("llm")
        self.graph = graph.compile()

    def should_continue(self, state: AgentState) -> bool:
        """Check if there are any tool calls to process"""
        # The key here is to verify if there are planned tool calls from llm,
        # so we take the last message and check if the LLM required tool calls
        last_message = state["messages"][-1]
        return hasattr(last_message, "tool_calls") and bool(last_message.tool_calls)

    def call_llm(self, state: AgentState) -> AgentState:
        """Process messages through the LLM"""
        messages = state["messages"]
        if self.system and not any(isinstance(m, SystemMessage) for m in messages):
            messages = [SystemMessage(content=self.system)] + messages
        response = self.model.invoke(messages)
        return {"messages": [response]}

    def take_action(self, state: AgentState) -> AgentState:
        """Execute tool calls and return results"""
        last_message = state["messages"][-1]
        results = []

        # Notice that the message include tool calls required by LLM
        # in the property tool_calls. We iterate the property and invoke the
        # tool.
        # The tool call could be simplified using a ToolNode, but this
        # code explains clearly what happens under the hood
        for tool_call in last_message.tool_calls:
            tool_name = tool_call['name']
            if tool_name not in self.tools:
                result = f"Error: Unknown tool '{tool_name}'"
            else:
                try:
                    tool_result = self.tools[tool_name].invoke(tool_call['args'])
                    result = str(tool_result)
                except Exception as e:
                    result = f"Error executing {tool_name}: {str(e)}"

            results.append(
                ToolMessage(
                    tool_call_id=tool_call['id'],
                    name=tool_name,
                    content=result
                )
            )

        return {"messages": results}

    def invoke(self, message: str) -> List[AnyMessage]:
        """Main entry point for the agent"""
        initial_state = {"messages": [HumanMessage(content=message)]}
        final_state = self.graph.invoke(initial_state)
        return final_state["messages"]

## Agent creation

Now, we create a function that creates the agent and the tools

In [12]:
def create_restaurant_agent() -> ReActAgent:
    model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    # Create tool instance
    restaurant_tool = RestaurantTool()

    # Convert to LangChain Tool
    tool = Tool(
        name=restaurant_tool.name,
        description=restaurant_tool.description,
        func=restaurant_tool
    )

    # Create system prompt
    system_prompt = prompt_template.format(
        tools=tool.description,
        tool_names=tool.name,
        input="{input}",
        agent_scratchpad="{agent_scratchpad}"
    )

    # Create and return agent
    return ReActAgent(model, [tool], system=system_prompt)

## Execution

Now we execute our agent

In [13]:
agent = create_restaurant_agent()
response = agent.invoke("""which resturant have better rating, Pizza Palace or Burger Barn?""")
for message in response:
    print(f"{message.type}: {message.content}")

human: which resturant have better rating, Pizza Palace or Burger Barn?
ai: 
tool: Rating: 4.5/5.0 from 230 reviews
tool: Rating: 4.2/5.0 from 185 reviews
ai: Final Answer: Pizza Palace has a better rating with 4.5 out of 5.0 compared to Burger Barn's rating of 4.2 out of 5.0.
